In [1]:

import numpy as np
import matplotlib
matplotlib.use('TkAgg')
import nimfa
import matplotlib.pyplot as plt
import scipy.sparse as spr
import cv2
import os


"""
    采用降维方法
    （1）图像分块
    （2）每块用非负矩阵分解寻找一组基图像
    （3）带压缩图片的每块分解到基图像上
"""

SPARE_ELEMENT = 0
SPARE_RATE = 0.5
# 边里该文件夹下的文件名称
def read_directory(directory_name):
    file_list = []
    for filename in os.listdir(directory_name):
        str = directory_name + '/' + filename
        file_list.append(str)
    return file_list

def bright_Norm(intput, l, h, w):
    """
        调整采集图片的亮度色度问题
        根据灰度、Gamma归一化亮度
    """
    normImg_list = np.zeros([l, h, w], np.uint8)  # 用于保存亮度变换后的图像

    id = 0
    for input_path in intput:
        # print("开始处理第?????", id, "个，其地址为：", input_path)
        image = cv2.imread(input_path, 0)  # 图片地址
        # print(image.shape)
        Gamma = np.log(128.0/255.0) / np.log(cv2.mean(image)[0]/255.0)
        lookUpTable = np.empty((1, 256), np.uint8)

        for i in range(256):
            lookUpTable[0, i] = np.clip(pow(i / 255.0, Gamma)*255.0, 0, 255)

        normImg_list[id] = cv2.LUT(image, lookUpTable)
        id += 1
        # cv2.imshow(filename, imageNorm)
        # cv2.waitKey(0)
        #
        # cv2.imwrite('./BriNorm_image'+'/'+filename, imageNorm)
    return normImg_list

def bright_Norm_one(image):

    Gamma = np.log(128.0/255.0) / np.log(cv2.mean(image)[0]/255.0)
    lookUpTable = np.empty((1, 256), np.uint8)

    for i in range(256):
        lookUpTable[0, i] = np.clip(pow(i / 255.0, Gamma)*255.0, 0, 255)

    imageNorm = cv2.LUT(image, lookUpTable)

    return imageNorm

# # 读取的目录
# bright_Norm('./Original_image')
def divide_method1(img, m, n):  # 分割成m行n列
    print(img.shape)
    h, w = img.shape[0], img.shape[1]
    gx = np.round(h).astype(np.int)
    gy = np.round(w).astype(np.int)
    divide_image = np.zeros([m - 1, n - 1, int(h * 1.0 / (m - 1) + 0.5), int(w * 1.0 / (n - 1) + 0.5), 3],
                            np.uint8)  # 这是一个五维的张量，前面两维表示分块后图像的位置（第m行，第n列），后面三维表示每个分块后的图像信息
    for i in range(m - 1):
        for j in range(n - 1):
            print(i)
            print(j)
            # 这样写比a[i,j,...]=要麻烦，但是可以避免网格分块的时候，有些图像块的比其他图像块大一点或者小一点的情况引起程序出错
            print(img[gy[i][j]:gy[i + 1][j + 1], gx[i][j]:gx[i + 1][j + 1], :])
            divide_image[i, j, 0:gy[i + 1][j + 1] - gy[i][j], 0:gx[i + 1][j + 1] - gx[i][j], :] = img[
                                                                                                  gy[i][j]:gy[i + 1][
                                                                                                      j + 1],
                                                                                                  gx[i][j]:gx[i + 1][
                                                                                                      j + 1], :]

    return divide_image


def divide_method2(img, m, n):  # 分割成m行n列
    h, w = img.shape[0], img.shape[1]
    grid_h = int(h * 1.0 / (m - 1) + 0.5)  # 每个网格的高
    grid_w = int(w * 1.0 / (n - 1) + 0.5)  # 每个网格的宽

    # 满足整除关系时的高、宽
    h = grid_h * (m - 1)
    w = grid_w * (n - 1)

    # 图像缩放
    img_re = cv2.resize(img, (w, h),
                        cv2.INTER_LINEAR)  # 也可以用img_re=skimage.transform.resize(img, (h,w)).astype(np.uint8)
    # plt.imshow(img_re)
    gx, gy = np.meshgrid(np.linspace(0, w, n), np.linspace(0, h, m))
    gx = gx.astype(np.int_)
    gy = gy.astype(np.int_)

    divide_image = np.zeros([m - 1, n - 1, grid_h, grid_w],
                            np.uint8)  # 这是一个五维的张量，前面两维表示分块后图像的位置（第m行，第n列），后面三维表示每个分块后的图像信息

    for i in range(m - 1):
        for j in range(n - 1):
            divide_image[i, j, ...] = img_re[
                                      gy[i][j]:gy[i + 1][j + 1], gx[i][j]:gx[i + 1][j + 1]]
    return divide_image


def isSparse(matrix):
    """
    Judge spare matrix.
    :param matrix: matrix
    :return: boolean
    """
    sum = len(matrix) * len(matrix[0])
    spare = 0

    for row in range(len(matrix)):
        for column in range(len(matrix[row])):
            if matrix[row][column] == SPARE_ELEMENT:
                spare += 1

    if spare / sum >= SPARE_RATE:
        return True
    else:
        return False

def save_blocks(title, divide_image):  #
    m, n = divide_image.shape[0], divide_image.shape[1]
    for i in range(m):
        for j in range(n):
            print(divide_image[i, j].shape)
            # plt.imshow(divide_image[i, j, :])
            # # plt.show()
            # plt.axis('off')
            # plotPath = str(title) + "+" + str(i) + str(j) + '.jpg'  # 图片保存路径
            # plt.imsave("./img_list/" + plotPath, divide_image[i, j, :], cmap='gray')  # 保存灰度图像


def nmf_ls(V, r):
    lsnmf = nimfa.Lsnmf(V, seed='random_vcol', max_iter=100, rank=r, track_error=True)  # LSNMF with Random VCol initialization and rank estimation.
    # lsnmf = nimfa.Lsnmf(V, max_iter=100, rank=r)
    # print(V.shape)
    lsnmf_fit = lsnmf()

    W = lsnmf_fit.basis()
    # print("W:", W.shape)
    # print('Basis matrix:\n%s' % W)

    H = lsnmf_fit.coef()
    # print("H:", H.shape)
    # print('Mixture matrix:\n%s' % H)
    #
    # print('K-L divergence: %5.3f' % lsnmf_fit.distance(metric='kl'))
    #
    # print('Rss: %5.3f' % lsnmf_fit.fit.rss())
    # print('Evar: %5.3f' % lsnmf_fit.fit.evar())
    # print('Iterations: %d' % lsnmf_fit.n_iter)
    # print('Target estimate:\n%s' % np.dot(W, H))
    return W, H


def nmf_standard(V, r):
    nmf = nimfa.Nmf(V, max_iter=200, rank=r, update='euclidean', objective='fro')
    nmf_fit = nmf()

    W = nmf_fit.basis()
    # print('Basis matrix:\n%s' % W.todense())

    H = nmf_fit.coef()
    # print('Mixture matrix:\n%s' % H.todense())
    #
    # print('Euclidean distance: %5.3f' % nmf_fit.distance(metric='euclidean'))

    sm = nmf_fit.summary()
    # print('Sparseness Basis: %5.3f  Mixture: %5.3f' % (sm['sparseness'][0], sm['sparseness'][1]))
    # print('Iterations: %d' % sm['n_iter'])
    # print('Target estimate:\n%s' % np.dot(W.todense(), H.todense()))
    return W, H

def create_substrate(intput, h, w, m, n, r):

    '''
    构建基底
    r 基中向量的个数
    划分原始图像并抽取每个子块的基
    :return:
    '''
    print('input shape:', intput.shape)
    grid_h = int(h * 1.0 / m)  # 每个网格的高
    grid_w = int(w * 1.0 / n)  # 每个网格的宽
    divide_image = np.zeros([bm, m, n, grid_h, grid_w],
                            np.uint8)  # 用于保存分割后的图像
    print('divide_image_shape:', divide_image.shape)

    # print("==============开始分块处理文件夹内的图片==============")
    # for input_path in intput:
        # print("开始处理第", title, "个，其地址为：", input_path)
        # img = cv2.imread(input_path, 0)  # 图片地址
        # img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    for title in range(intput.shape[0]):
        img = intput[title]
        divide_image2 = divide_method2(img, m + 1, n + 1)  # 该函数中m+1和n+1表示网格点个数，m和n分别表示分块的块数
        # save_blocks(title, divide_image2)
        divide_image[title] = divide_image2
        # print(divide_image[title-1, ...].shape)
        # print(divide_image2.shape)
        # print("第", title, "个已分块完毕")
        # print("-----------------------------")
        title += 1
    # print("==============完成全部分块处理文件夹的图片==============")

    # (2) 抽取
    X = np.zeros([m, n, bm, grid_h * grid_w], np.uint8)  # 存储每一块抽取基矩阵的输入向量 [2,6,249,262144]
    Basis = np.zeros([m, n, r, grid_h * grid_w], np.uint8)  # 存储每一块的基矩阵[2, 6, 100, 262144]
    # print('X.shape:', X.shape)
    for i in range(m):
        for j in range(n):
            for k in range(bm):
                X[i][j][k] = divide_image[k, i, j, ...].flatten()
            # print('X[i][j].shape:', X[i][j].shape)  # [249,262144]
            W, H = nmf_ls(X[i][j], r)  # H.T 是基矩阵
            # print('H shape:', H.shape)  # [100, 262144]
            Basis[i, j] = H

    return Basis

def compute_coef( testimg, H, l, m, n):
    '''
    计算测试图片用基矩阵表示的向量
    :param testimg: 测试图像集合
    :param m: 纵向划分几块
    :param n: 横向划分几块
    :return: 系数向量
    '''
    id = 1
    for test_path in testimg:
        # print("开始处理第", id, "个，其地址为：", test_path)
        img = cv2.imread(test_path, 0)  # 图片地址
        # cv2.imshow(str(id), img)
        # cv2.waitKey(0)

        # （1）亮度变换
        norm_img = bright_Norm_one(img)
        # cv2.imshow(str(id), norm_img)
        # cv2.waitKey()

        # （2）划分图像
        divide_image2 = divide_method2(norm_img, m + 1, n + 1)  # 该函数中m+1和n+1表示网格点个数，m和n分别表示分块的块数
        # print(divide_image2.shape)

        # （3）用基底表示图
        cn = np.zeros([l, m, n, 100, 1], np.float64)
        # cn.reshape((l, m, n, 100))
        for i in range(divide_image2.shape[0]):
            for j in range(divide_image2.shape[1]):
                # print(i, j)
                d_pre = divide_image2[i, j].flatten()
                y = d_pre.reshape(-1, 1)
                # print(y.shape)
                # 把公式写出来
                # print("先看看", np.linalg.inv((H.T) * H) * (H.T) * y)

                ci = np.linalg.inv((H.T) * H) * (H.T) * y  # 系数向量c
                # print("ci???????", ci.shape)
                # print(ci)
                cn[id, i, j] = ci
                # print("leixing:", cn.dtype, ci.dtype)
                # print(cn[id, i, j].shape)
                # print(cn[id, i, j])
    id += 1
    return cn











In [2]:
# def compute_coef_one(img, Basis, m, n, r):
#     # （1）亮度变换
#     norm_img = bright_Norm_one(img)
#     # cv2.imshow(str(id), norm_img)
#     # cv2.waitKey(0)

#     # （2）划分图像
#     divide_image2 = divide_method2(norm_img, m + 1, n + 1)  # 该函数中m+1和n+1表示网格点个数，m和n分别表示分块的块数
#     # print(divide_image2.shape)

#     # （3）用基底表示图
#     cn = np.zeros([m, n, r, 1], np.float64)
#     for i in range(m):
#         for j in range(n):
#             # print(i, j)
#             # cv2.imshow('sub', divide_image2[i, j])
#             # cv2.waitKey(0)
#             d_pre = divide_image2[i, j].flatten()
#             y = d_pre.reshape(-1, 1)
#             print(y.shape)
#             # 把公式写出来
#             # print("先看看", np.linalg.inv((H.T) * H) * (H.T) * y)
#             ci = np.dot(np.dot(np.linalg.inv(np.dot(Basis[i, j], Basis[i, j].T)), Basis[i, j]), y)  # 系数向量c
#             # ci = np.linalg.inv((Basis[i, j].T) * Basis[i, j]) * (Basis[i, j].T) * y  # 系数向量c

#             cn[i, j] = ci

#     #  （4）计算残差
#     residuals = np.zeros([m, n, 512*512, 1], np.float64)
#     for i in range(divide_image2.shape[0]):
#         for j in range(divide_image2.shape[1]):
#             d_pre = divide_image2[i, j].flatten()
#             y = d_pre.reshape(-1, 1)
#             # print('打印出来看看', residuals[i, j].shape, y.shape, np.dot(Basis[i, j].T, cn[i, j]).shape)
#             cv2.imshow('image', np.dot(Basis[i, j].T, cn[i, j]).reshape(512, 512))
#             cv2.waitKey(0)
#             residuals[i, j] = y - np.dot(Basis[i, j].T, cn[i, j])
#     print('residuals.shape', residuals.shape)
#     return cn, residuals

In [3]:


# 读取图像
intput = read_directory("./Original_image")
print("图片共有：", len(intput))
bm = len(intput)  # 原始维度
h, w = 1024, 3072
m = 2 # 8
n = 6  # 24
r = 10
norm_img_list = bright_Norm(intput, bm, h, w)

# 划分原始图像并抽取每个子块的基
Basis = create_substrate(norm_img_list, h, w, m, n, r)

print('Basis.shape:', Basis.shape)
# [2,6,100, 512*512]
# for i in range(m):
#     for j in range(n):
#         for k in range(r):
#             cv2.imshow('basis', Basis[i, j, k].reshape(512, 512))
#             cv2.waitKey(0)
# print(Basis)

   

    # 对于未知子图，用基表示图像，寻找系数向量c
    # 处理一个目录
    # testimg = read_directory("./test")  # 读取图片
    # print(len(testimg))
    # c = compute_coef(testimg, bm, m, n)
    # print(c.shape)
    # print(c)
    # print(c[1, 1, 2])


图片共有： 249
input shape: (249, 1024, 3072)
divide_image_shape: (249, 2, 6, 512, 512)
Basis.shape: (2, 6, 10, 262144)


In [21]:
 # 处理单张图片
img = cv2.imread('000002.jpg', 0)
print(img.shape)


# （1）亮度变换
norm_img = bright_Norm_one(img)
# cv2.imshow(str(id), norm_img)
# cv2.waitKey(0)

# （2）划分图像
divide_image2 = divide_method2(norm_img, m + 1, n + 1)  # 该函数中m+1和n+1表示网格点个数，m和n分别表示分块的块数
# print(divide_image2.shape)
# （3）用基底表示图
cn = np.zeros([m, n, r, 1], np.float64)
for i in range(m):
    for j in range(n):
        # print(i, j)
        # cv2.imshow('sub', divide_image2[i, j])
        # cv2.waitKey(0)
        d_pre = divide_image2[i, j].flatten()
        y = d_pre.reshape(-1, 1)
        print(y.shape)
        # 把公式写出来
        # print("先看看", np.linalg.inv((H.T) * H) * (H.T) * y)
        X = Basis[i, j].T  # [262144, 100]
        print('-------------------------基---------------------------')
        print(X.shape)
        print(X)
        print('-------------------------XTX---------------------------')
        print(np.dot(X.T, X).shape)
        print(np.dot(X.T, X))
        print('-------------------------求拟---------------------------')
        print(np.linalg.pinv(np.dot( X.T, X)).shape)
        print(np.linalg.pinv(np.dot(X.T, X)))
        print(np.linalg.pinv(np.dot(X.T, X)))
        ci = np.dot(np.dot(np.linalg.pinv(np.dot(X.T, X)), X.T), y)  # 系数向量c
        print('-------------------------系数---------------------------')
        print(ci.shape)
        print(ci)

              
#         ci = np.dot(np.dot(np.linalg.pinv(np.dot(Basis[i, j], Basis[i, j].T)), Basis[i, j]), y)  # 系数向量c
        # ci = np.linalg.inv((Basis[i, j].T) * Basis[i, j]) * (Basis[i, j].T) * y  # 系数向量c



        cn[i, j] = ci



(1024, 3072)
(262144, 1)
-------------------------基---------------------------
(262144, 10)
[[ 93  97  94 ...  98  95  98]
 [ 93  97  94 ...  98  94  98]
 [ 93  96  94 ...  97  94  97]
 ...
 [120 122 119 ... 118 114 117]
 [120 122 118 ... 118 114 116]
 [119 121 118 ... 118 114 116]]
-------------------------XTX---------------------------
(10, 10)
[[133  61 138 218 203 215  27 196 200 150]
 [ 61 212 158  19  34 225  92 254 108 122]
 [138 158  17 229 167 243  66 150 252 189]
 [218  19 229 220 169  62 219 110 169 250]
 [203  34 167 169 164 223 245 139 226   7]
 [215 225 243  62 223  21  19  35  22 249]
 [ 27  92  66 219 245  19 113 105  20 248]
 [196 254 150 110 139  35 105 118 199 226]
 [200 108 252 169 226  22  20 199 252  89]
 [150 122 189 250   7 249 248 226  89 117]]
-------------------------求拟---------------------------
(10, 10)
[[ 2.65144851e-01 -7.22009050e-02 -2.03072821e-01 -7.58129229e-02
   3.27319919e-02 -5.06427930e-02 -2.59050072e-03  2.19719222e-01
  -1.26933343e-01  8.843

-------------------------系数---------------------------
(10, 1)
[[-10039172.2398819 ]
 [-20380043.2722487 ]
 [ 24848694.52948065]
 [ 25779182.81046803]
 [ 83558180.58926025]
 [-87191804.99035367]
 [ 12534498.05095769]
 [-98413149.43683708]
 [ 15374224.02401828]
 [ 34635941.42397824]]
(262144, 1)
-------------------------基---------------------------
(262144, 10)
[[127 125 129 ... 131 130 124]
 [127 125 128 ... 131 130 124]
 [127 125 128 ... 131 130 124]
 ...
 [109 120 102 ... 107 103 124]
 [113 124 105 ... 111 107 128]
 [114 124 107 ... 113 110 131]]
-------------------------XTX---------------------------
(10, 10)
[[  9  63  83 103  73 166  64 186  93   5]
 [ 63  14 224 198 221 254 204  50 252  86]
 [ 83 224 171  83  58  30 246  20 179 214]
 [103 198  83  59 174  25 196 149 237 145]
 [ 73 221  58 174 200 191 173  61 176 188]
 [166 254  30  25 191  20 249  65  80 200]
 [ 64 204 246 196 173 249  99 123 170  72]
 [186  50  20 149  61  65 123  12  29 150]
 [ 93 252 179 237 176  80 170  29 12

-------------------------系数---------------------------
(10, 1)
[[  8141731.23629974]
 [-10367627.63845716]
 [  7336106.33240269]
 [  6894957.71115409]
 [  4227965.61189169]
 [ -1639232.61466787]
 [ -2235497.96755154]
 [  6580750.88481241]
 [   111142.36180255]
 [ 10182905.12520566]]
(262144, 1)
-------------------------基---------------------------
(262144, 10)
[[ 96  95  95 ...  96  94  95]
 [ 95  95  94 ...  95  94  95]
 [ 95  95  94 ...  94  93  95]
 ...
 [118 115 115 ... 117 120 119]
 [118 115 114 ... 118 120 119]
 [118 117 114 ... 118 120 119]]
-------------------------XTX---------------------------
(10, 10)
[[ 27 126 159  80   2 151  80 128 236 238]
 [126 146 206 100 187 249 119 242 140 223]
 [159 206  81  88 146 129  42 130 136 243]
 [ 80 100  88 231 128 127   5 182 120 247]
 [  2 187 146 128 233  58 239 252  95  22]
 [151 249 129 127  58 244 233 163 137 213]
 [ 80 119  42   5 239 233  87  86  92 230]
 [128 242 130 182 252 163  86 150 203 189]
 [236 140 136 120  95 137  92 203  2

[[-3.85163979e-03  2.45097815e-03  5.03957803e-03 -1.28967007e-03
   4.89534777e-03  1.24934950e-03 -1.69861153e-03 -3.02882728e-03
   5.46640019e-04 -4.58701476e-03]
 [ 2.45097815e-03 -8.99165383e-04 -2.96363424e-03 -2.96477910e-03
   9.18069689e-04  2.29154099e-03  3.39585761e-03 -1.04936430e-04
  -1.75092616e-03  1.38709767e-03]
 [ 5.03957803e-03 -2.96363424e-03  1.81336317e-03  3.72517658e-03
   1.72402134e-03 -4.84148147e-03  2.27093205e-03 -3.08360169e-03
  -1.65430795e-03 -8.13969434e-04]
 [-1.28967007e-03 -2.96477910e-03  3.72517658e-03  6.39868698e-03
  -3.96166233e-03 -2.49281588e-03 -5.75136807e-04 -1.11458111e-03
  -9.65044445e-04  1.58433926e-03]
 [ 4.89534777e-03  9.18069689e-04  1.72402134e-03 -3.96166233e-03
   2.83641228e-04 -2.59641226e-03  2.12220851e-03  7.46755396e-04
  -3.57697204e-03  1.31746079e-03]
 [ 1.24934950e-03  2.29154099e-03 -4.84148147e-03 -2.49281588e-03
  -2.59641226e-03  2.64745517e-03  3.84111099e-03  3.00904773e-05
   8.52542560e-04 -3.80227374e-04

[[ 91 238 252 144 110 222 178 164  85 219]
 [238 204 241 181  95 170  52 194 126 119]
 [252 241 179  91  93 244 237 249 189 151]
 [144 181  91  12 197  40 209 180  79 154]
 [110  95  93 197  47  36 116 188  37  14]
 [222 170 244  40  36  64 155 228 247 215]
 [178  52 237 209 116 155  57  79  51 190]
 [164 194 249 180 188 228  79 113 112 240]
 [ 85 126 189  79  37 247  51 112  15 245]
 [219 119 151 154  14 215 190 240 245 205]]
-------------------------求拟---------------------------
(10, 10)
[[ 0.06720978  0.09974138 -0.06494353  0.05337145 -0.07984762 -0.05152114
   0.02751584 -0.099634   -0.02947534  0.06390062]
 [ 0.09974138  0.1367455  -0.09451124  0.07143964 -0.10723058 -0.0702875
   0.02751907 -0.13062796 -0.04189477  0.08855007]
 [-0.06494353 -0.09451124  0.0679273  -0.05295018  0.07444625  0.05248335
  -0.01845239  0.08994776  0.02758347 -0.06731145]
 [ 0.05337145  0.07143964 -0.05295018  0.03734683 -0.05454932 -0.03902051
   0.01697626 -0.06866783 -0.02400113  0.05045156]
 [-0.0

In [23]:
#  （4）计算残差
residuals = np.zeros([m, n, 512*512, 1], np.float64)
for i in range(divide_image2.shape[0]):
    for j in range(divide_image2.shape[1]):
        d_pre = divide_image2[i, j].flatten()
        y = d_pre.reshape(-1, 1)
        # print('打印出来看看', residuals[i, j].shape, y.shape, np.dot(Basis[i, j].T, cn[i, j]).shape)
        print('-----------------基础 * 系数------------------')
        print( np.dot(Basis[i, j].T, cn[i, j]).reshape(512, 512))
        cv2.imshow('image', np.dot(Basis[i, j].T, cn[i, j]).reshape(512, 512))
        cv2.waitKey(0)
        residuals[i, j] = y - np.dot(Basis[i, j].T, cn[i, j])
print('residuals.shape', residuals.shape)
#     return cn, residuals
# print(c.shape)
# print(c)


# for i in range(m):
#     for j in range(n):
#         cv2.imshow('1', residuals[i, j].reshape(512, 512))
#         cv2.waitKey(0)
#         print(isSparse(residuals[i, j]))



-----------------基础 * 系数------------------
[[1.88607123e+09 1.88520647e+09 1.88451479e+09 ... 2.28773503e+09
  2.29202818e+09 2.30563621e+09]
 [1.87952980e+09 1.88417305e+09 1.87541787e+09 ... 2.28031642e+09
  2.27244986e+09 2.27759560e+09]
 [1.88630544e+09 1.89568179e+09 1.88304610e+09 ... 2.27709310e+09
  2.25960456e+09 2.28992457e+09]
 ...
 [1.86797936e+09 1.86956831e+09 1.87205117e+09 ... 2.33230775e+09
  2.33141875e+09 2.34229257e+09]
 [1.86536648e+09 1.87099473e+09 1.87424033e+09 ... 2.32702960e+09
  2.32188385e+09 2.33504518e+09]
 [1.89077456e+09 1.88025083e+09 1.88562882e+09 ... 2.32248155e+09
  2.30842593e+09 2.31173261e+09]]
-----------------基础 * 系数------------------
[[-1.59470193e+09 -1.62297644e+09 -1.72361448e+09 ... -2.60632733e+09
  -2.55282605e+09 -2.51648868e+09]
 [-1.55855194e+09 -1.66700426e+09 -1.69781679e+09 ... -2.47034427e+09
  -2.52518887e+09 -2.35188295e+09]
 [-1.56274893e+09 -1.69579802e+09 -1.63293302e+09 ... -2.73254343e+09
  -2.40774923e+09 -2.47578851e+09]